# Crear un TorchPlayer


Recibe el modelo a instanciar como path y juega con el mismo

- Pensar como resolver el problema de que solo samplee las válidas
- Agregarle la opción de monte carlo tree search (opcional) con las opciones de iterationLimit, timeLimit

Si va a agregar MCTS mirar la notebook 007_MCTS.ipnb

## Aclaraciones para docentes:
TorchPlayer está tambien en players.py pero la copio acá para mayor claridad de la notebook. Hereda de BasePlayer, clase que está definida en players.py y contiene una interfaz común que utilizo en clases como Arena (arena.py) y Tournament (tournament.py), para jugar partidos. Además posee un par de métodos extra por el mismo motivo.

Reescribí monte carlo tree search en un archivo nuevo (multi_process_mcts.py) e hice los siguientes cambios:
* MultiProcessMonteCarlo:
    * Nueva clase que puede correr muchas simulaciones en paralelo utilizando múltiples procesos. 
    * Además cambia el sistema de límites para ser por profundidad (levelLimit) en vez de iteraciones o tiempo. 
        * Si el nivel de profundidad (levelLimit) es 1, por cada step toma todos los siguientes posibles nodos en base a acciones validas (por lo general 3 o 4) y juega una partida contra si mismo en cada uno utilizando el modelo actual, eligiendo luego la mejor opción. 
        * Si es 2, toma todos los hijos de los hijos y juega todas esas posibilidades, y luego en base a un reward ponderado por visitas elige la mejor acción.
    * El motivo es que correr tantas simulaciones para tomar cada decisión era muy lento, y por eso al utilizarlo con levelLimit=1 y multiples procesos se le da la misma importancia a cada rama del árbol y se corren 3 o 4 juegos en paralelo por movida, acelerando mucho el desarrollo del partido.
* CustomReversiState => Reversión de ReversiState con algunos cambios, por ej no hacer deep cloning del env.

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from typing import Union, Optional, Dict, Any
from boardgame2 import BoardGameEnv
from stable_baselines3 import PPO
from multi_process_mcts import MultiProcessMonteCarlo, model_policy
from reversi_state import CustomReversiState
from players import BasePlayer
import os

In [5]:
class TorchPlayer(BasePlayer):
    def __init__(self,
                 player: int = 1,
                 env: BoardGameEnv = None,
                 flatten_action: bool = False,
                 model_path: str = None,
                 deterministic: bool = True,
                 only_valid: bool = True,
                 mcts: bool = False,
                 levelLimit: int = None,
                 device: str = 'auto',
                 mtcs_n_processes: int = None
                 ):

        if model_path is None:
            raise Exception("model_path cannot be None")

        super().__init__(player, env, flatten_action, os.path.splitext(os.path.basename(model_path))[0])

        self.model = PPO.load(model_path, device=device)
        self.model_path = model_path
        self.deterministic = deterministic
        self.only_valid = only_valid
        self.mcts = mcts
        self.levelLimit = levelLimit
        self.mtcs_n_processes = mtcs_n_processes

    def predict(self, board: np.ndarray) -> Union[int, np.ndarray]:
        if self.mcts:
            action = self._get_action_with_mcts(board)
            action = action.action
            if self.flatten_action:
                return action[0] * self.board_shape + action[1]
            else:
                return action
        else:
            obs = self.player * board
            if self.only_valid:
                obs = [obs, self.env.get_valid((obs, 1))]
            # The model expects a batch of observations.
            # Make a batch of 1 obs
            obs = [obs]
            action = self.model.predict(obs, deterministic=self.deterministic)[0]

            if self.flatten_action:
                return action
            else:
                return np.array([action // self.board_shape, action % self.board_shape])

    def _get_action_with_mcts(self, board: np.ndarray) -> Union[int]:
        searcher = MultiProcessMonteCarlo(levelLimit=self.levelLimit,
                                          n_processes=self.mtcs_n_processes,
                                          explorationConstant=0.0,
                                          rolloutPolicy=model_policy(self.model))

        state = CustomReversiState(self.env, (board, self.player))
        return searcher.search(initialState=state)

    def __str__(self):
        monte_carlo = f"- MCTS" if self.mcts else ""
        return f"{self.__class__.__name__}({self.name}{monte_carlo})"

# Arena

Testear el jugador contra los distintos jugadores

#### Aclaraciones para docentes: En arena.py se puede encontrar una implementación de una clase "Arena" que permite correr partidas entre 2 jugadores de cualquier clase, imprimir resultados y almacenar estadísticas. Utilizo el mejor modelo para ver los resultados obtenidos.

In [6]:
from arena import Arena
from players import RandomPlayer, GreedyPlayer, TorchPlayer
from boardgame2 import ReversiEnv

In [7]:
env = ReversiEnv(board_shape=8)

## Torch vs Random:

In [8]:
player_1 = TorchPlayer(player=1, env=env, model_path="./Vs_Multiple_v2.zip")
player_2 = RandomPlayer(player=-1, env=env)
arena = Arena(player_1, player_2, env, verbose=True)

In [9]:
arena.play(n_games=100)


MATCH: TorchPlayer(Vs_Multiple_v2) vs RandomPlayer

Playing n:100/100 	 Wins(player 1/ player 2):92.93%/2.02% 	 Ties:5.05%

THE WINNER IS TorchPlayer(Vs_Multiple_v2)!



In [10]:
arena.print_players_stats()


####### STATS FOR PLAYER: 1 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.8775510204081632
Wins as second: 0.9803921568627451
Ties: 0.05
Plays as first: 49
Plays as second: 51
Avg game duration: 59.61

#################################################################
            
        

####### STATS FOR PLAYER: 2 - RandomPlayer #######

Wins as first: 0.0196078431372549
Wins as second: 0.02040816326530612
Ties: 0.05
Plays as first: 51
Plays as second: 49
Avg game duration: 59.61

##################################################
            
        


#### Gana cerca del 88% como primero y 98% como segundo.

## Torch vs Greedy:

In [11]:
player_1 = TorchPlayer(player=1, env=env, model_path="./Vs_Multiple_v2.zip")
player_2 = GreedyPlayer(player=-1, env=env)
arena = Arena(player_2, player_1, env, verbose=True)

In [12]:
arena.play(n_games=100)


MATCH: GreedyPlayer vs TorchPlayer(Vs_Multiple_v2)

Playing n:100/100 	 Wins(player 1/ player 2):6.06%/92.93% 	 Ties:1.01%

THE WINNER IS TorchPlayer(Vs_Multiple_v2)!



In [13]:
arena.print_players_stats()


####### STATS FOR PLAYER: 1 - GreedyPlayer #######

Wins as first: 0.11538461538461539
Wins as second: 0.0
Ties: 0.01
Plays as first: 52
Plays as second: 48
Avg game duration: 58.81

##################################################
            
        

####### STATS FOR PLAYER: 2 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.9791666666666666
Wins as second: 0.8846153846153846
Ties: 0.01
Plays as first: 48
Plays as second: 52
Avg game duration: 58.81

#################################################################
            
        


#### Se comporta similar que vs Random. Juega mejor como segundo.

## Torch vs Torch:

### No deterministico: Más interesante, sino juega siempre el mismo partido

In [14]:
player_1 = TorchPlayer(player=1, env=env, model_path="./Vs_Multiple_v2.zip", deterministic=False)
player_2 = TorchPlayer(player=-1, env=env, model_path="./Vs_Multiple_v2.zip", deterministic=False)
arena = Arena(player_1, player_2, env, verbose=True)

In [15]:
arena.play(n_games=100)


MATCH: TorchPlayer(Vs_Multiple_v2) vs TorchPlayer(Vs_Multiple_v2)

Playing n:100/100 	 Wins(player 1/ player 2):54.55%/42.42% 	 Ties:3.03%

THE WINNER IS TorchPlayer(Vs_Multiple_v2)!



In [16]:
arena.print_players_stats()


####### STATS FOR PLAYER: 1 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.48148148148148145
Wins as second: 0.6304347826086957
Ties: 0.03
Plays as first: 54
Plays as second: 46
Avg game duration: 59.9

#################################################################
            
        

####### STATS FOR PLAYER: 2 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.32608695652173914
Wins as second: 0.5
Ties: 0.03
Plays as first: 46
Plays as second: 54
Avg game duration: 59.9

#################################################################
            
        


#### Esta parejo, jugando tanto de primero como segundo, que es lo esperado.

## Deterministico:

In [17]:
player_1 = TorchPlayer(player=1, env=env, model_path="./Vs_Multiple_v2.zip")
player_2 = TorchPlayer(player=-1, env=env, model_path="./Vs_Multiple_v2.zip")
arena = Arena(player_1, player_2, env, verbose=True)

In [18]:
arena.play(n_games=100)


MATCH: TorchPlayer(Vs_Multiple_v2) vs TorchPlayer(Vs_Multiple_v2)

Playing n:100/100 	 Wins(player 1/ player 2):54.55%/45.45% 	 Ties:0.0%

THE WINNER IS TorchPlayer(Vs_Multiple_v2)!



In [19]:
arena.print_players_stats()


####### STATS FOR PLAYER: 1 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.0
Wins as second: 1.0
Ties: 0.0
Plays as first: 46
Plays as second: 54
Avg game duration: 60.0

#################################################################
            
        

####### STATS FOR PLAYER: 2 - TorchPlayer(Vs_Multiple_v2) #######

Wins as first: 0.0
Wins as second: 1.0
Ties: 0.0
Plays as first: 54
Plays as second: 46
Avg game duration: 60.0

#################################################################
            
        


#### Se gana a si mismo el 100% si arranca segundo. Esto es simplemente porque al ser los 2 deterministicos juegan el mismo juego una y otra vez.

## Torch vs Torch con Monte Carlo: 

#### Jugar con Monte Carlo toma mucho tiempo. Pruebo con 1 solo nivel de profundidad y a 50 juegos. Modo No deterministico.

In [20]:
player_1 = TorchPlayer(player=1, env=env, model_path="./Vs_Multiple_v2.zip", deterministic=False)
player_2 = TorchPlayer(player=-1, env=env, model_path="./Vs_Multiple_v2.zip", deterministic=False, mcts=True, levelLimit=1, device="cpu")
arena = Arena(player_1, player_2, env, verbose=True)

In [21]:
arena.play(n_games=50)


MATCH: TorchPlayer(Vs_Multiple_v2) vs TorchPlayer(Vs_Multiple_v2- MCTS)

Playing n:50/50 	 Wins(player 1/ player 2):20.41%/73.47% 	 Ties:6.12%%

THE WINNER IS TorchPlayer(Vs_Multiple_v2- MCTS)!



#### Se puede ver que player 2 (con Monte Carlo) fue bastánte superior a sin usarlo (casi 75% vs 20%)

### Aclaración: En la notebook 010_Torneo hay varios match entre distintos modelos entrenados de diferentes maneras